In [27]:
import requests, re, time
from bs4 import BeautifulSoup
from selenium import webdriver
from forex_python.converter import CurrencyRates
import datetime
from db import Database

save_db = Database()

# Search parameters
driver = webdriver.Chrome('./chromedriver.exe')
driver.implicitly_wait(2)
driver.maximize_window()

# Search Page 
url = "https://www.seoulauction.com/search?lang=ko"
artist = '김환기'
ID = 'jeihyunsung'
PW = 'wpwp3008!'

# Login Task

driver.get(url)
driver.implicitly_wait(2)

driver.find_element_by_xpath('//*[@id="header"]/div[3]/ul/li[3]/a').click()

driver.find_element_by_xpath('//*[@id="loginId"]').send_keys(ID)
driver.implicitly_wait(2)
driver.find_element_by_xpath('//*[@id="password"]').send_keys(PW)
driver.implicitly_wait(2)
driver.find_element_by_xpath('//*[@id="loginForm"]/fieldset/div/div[3]/div[3]/span/button').click()
driver.implicitly_wait(2)

# search artist, button click
driver.find_element_by_xpath('//*[@id="container"]/div[3]/div/form/fieldset/div/div[1]/div[2]/dl[1]/dd/div/input[1]').send_keys(artist)
driver.implicitly_wait(2)
driver.find_element_by_css_selector('#container > div.contents_wrap > div > form > fieldset > div > div.btn_wrap > span.btn_style01.gray02 > button').click()
driver.implicitly_wait(2)
pages = int(driver.find_element_by_css_selector('#container > div.contents_wrap > div > form > div > div.left > div > paging > ul > li:nth-child(12) > a').text)
print("pages:", pages)
page = 0

element_id = 0

while(page < pages):
    print("get page : ", page)
    art_list = driver.find_elements_by_xpath('//*[@id="auctionList"]/li')
    for idx, element in enumerate(art_list):
        explain = element.find_element_by_class_name("auction_list_explain")
        description = explain.text

        try:
            size = explain.find_element_by_xpath('.//*[@ng-repeat="size in lot.LOT_SIZE_JSON"]').text
        except:
            size = 0
        try:
            method = explain.find_element_by_xpath('.//*[@ng-bind="lot.MATE_NM_EN"]').text
        except:
            method = ''
        try:
            year = explain.find_element_by_xpath('.//*[@ng-if="lot.MAKE_YEAR_JSON[locale]"]').text
        except:
            year = 0
        try:
            final_price = element.find_element_by_class_name("txt_impo").text
        except:
            final_price = ''
        
        # print(size, method, year, final_price, sep='/')

        st_price_txt = element.find_element_by_class_name("es_price_right").text
        auction_ect = element.find_element_by_class_name("auction_ect").text

        auction_ect = auction_ect.split('[')[1]
        auction_ect = auction_ect[:-1].split(' ')[0]
        auction_ect = auction_ect.split('.')
        auction_year, auction_month, auction_date = auction_ect[0], auction_ect[1], auction_ect[2][:-3] 

        # description = description.split('\n')
        # if len(description) == 3:
        #     year, method, size = description[0], description[1], description[2]
        # elif len(description) == 2:
        #     method, size = description[0], description[1]
        #     year = 0
        # elif len(description) == 1:
        #     method = description[0]
        #     year, size = 0, 0
        # elif len(description) == 0:
        #     size, year, method = 0, 0, ''

        # Year parsing
        if type(year) == str:            
            if "mid" in year:
                year = year.replace("’", '').replace("s", '').replace("'", '').replace("mid", '').replace('-','')
                year = int(year)
            elif "early" in year:
                year = int(year.replace("’", '').replace("s", '').replace("early ", ''))+4
            elif "-" in year:
                st = year.split('-')[0]
                ed = year.split('-')[1]
                if len(ed) == 2:
                    year = int(st[:2]+ed)
                else:
                    year = int(ed.replace('s',''))
            elif '.' in year:
                year = int(year.split('.')[0])
            elif "년대" in year:
                if "년대 초반" in year:
                    year = int(year.replace("년대 초반", "")) + 4
                elif "년대 초" in year:
                    year = int(year.replace("년대 초", "")) + 4
                elif "년대 후반" in year:
                    year = int(year.replace("년대 후반", "")) + 9
                else:
                    year = int(year.replace("년대", "")) + 9
            elif "’" in year or "s" in year or "'s" in year or "’s" in year:
                year = year.replace("’", '').replace("s", '').replace("'", '')
                year = int(year)+9
            else:
                try:
                    year = int(year)
                except:
                    year = year
                    print("## Year parsing Problem ##", year)

        # Size parsing
        if type(size) == str:
            if "cm" in size:
                sizeidx = size.index('cm')
                size = size[:sizeidx]
                size = size.replace("cm", '')
                x, y = float(size.split('☓')[0]), float(size.split('☓')[0])
                area = x*y
            else:
                print("## Size parsing problem", size)
                area = 0

        # Price parsing
        if "추정가 별도문의" in st_price_txt:
            st_price = 0
        elif "KRW" in st_price_txt:
            st_price_list = st_price_txt.split('\n')
            for x in st_price_list:
                if "KRW" in x: 
                    st_price = x.replace('KRW ','').replace(',', '')
                    st_price = int(st_price)
        elif "HKD" in st_price_txt:
            st_price = st_price_txt.split('\n')[0]
            st_price = st_price.replace('HKD ','').replace(',', '')
            st_price = int(st_price)
            c = CurrencyRates()
            dateobj = datetime.date(int(auction_year), int(auction_month), int(auction_date))
            try:
                st_price = c.convert('HKD', 'KRW', st_price, dateobj)
            except:
                st_price = c.convert('HKD', 'KRW', st_price)
        else:
            print("## Price parsing problem ", st_price_txt)
            st_price = 0
        
        # final price parsing

        if final_price == '':
            final_price = st_price
        else:
            if "KRW" in final_price:
                final_price = int(final_price.replace('낙찰가 ', '').replace('KRW ', '').replace(',',''))
            elif "HKD" in final_price:
                final_price = int(final_price.replace('낙찰가 ', '').replace('HKD ', '').replace(',',''))
                c = CurrencyRates()
                dateobj = datetime.date(int(auction_year), int(auction_month), int(auction_date))
                try:
                    st_price = c.convert('HKD', 'KRW', final_price, dateobj)
                except:
                    st_price = c.convert('HKD', 'KRW', final_price)
            else:
                print("## Final Price parsing problem ", final_price)

        # print(year, method, size, st_price, auction_year, auction_month, auction_date, sep='/')

        # Image extraction
        # image = element.find_element_by_tag_name('img')
        # image_src = image.get_attribute('ng-src')
        # print(image_src)
        # image_link = "https://www.seoulauction.com/" + image_src
        # target_image = requests.get(image_link)
        # with open(f'images/filename_{element_id}.jpg', 'wb') as file:
        #     file.write(target_image.content)

        data = [page, index, year, method, area, st_price, final_price, auction_year, auction_month, auction_date]
        save_db.saveID(element_id, data)
        element_id += 1


    driver.find_element_by_xpath("//li[@title='Next Page']/a").click()
    page += 1
    time.sleep(5)

pages: 22
get page :  0


NameError: name 'index' is not defined

In [ ]:
# //*[@id="container"]/div[3]/div/form/fieldset/div/div[1]/div[2]/dl[1]/dd/div/input[1]
#container > div.contents_wrap > div > form > fieldset > div > div.btn_wrap > span.btn_style01.gray02 > button


# resp = requests.get(url, data={'searchContent':artist})
# html_src = resp.text
# soup = BeautifulSoup(html_src, 'html.parser')

# soup
#container > div.contents_wrap > div > form > fieldset > div > div.search_box > div:nth-child(2) > dl:nth-child(1) > dd > div > input:nth-child(1)

# subway_image = soup.select('#mw-context-text > div > table:nth-child(3) > tbody > tr:nth-child(2) > td > a > img')
#mw-content-text > div.mw-parser-output > table:nth-child(3) > tbody > tr:nth-child(1) > td > a > img

# subway_image = soup.select('tr > td > a > img')
# print(subway_image)

# xpath, 조금 더 간결하게 접근이 가능함
# soup.XPath('//*[@id="mw-content-text"]/div[1]/table[1]/tbody/tr[1]/td/a/img')

# #siteNotice : ID selector .mw-headline class를 의미 tag와 함께 사용할 수 있음

# id_selector = soup.select('#siteNotice')
# print(id_selector)


#auctionList > li:nth-child(5) > div.result_auction_img_wrap.ng-scope > div > a > span > img

#container > div.contents_wrap > div > form > div > div.left > div > paging > ul > li:nth-child(14) > a

In [57]:
import datetime
c = CurrencyRates()
dateobj1 = datetime.date(int(auction_year), int(auction_month), int(auction_date))
print(dateobj1)
dateobj = datetime.date(2014, 5, 23)
print(dateobj)
st_price = c.convert('USD', 'INR', 10, dateobj1)
# st_price = c.convert('HKD', 'KRW', st_price, dateobj)
print(st_price)

# res = c.convert(base_cur=currency,dest_cur='DKK',amount=amount,date_obj=dateobj)

2017-11-26
2014-05-23


RatesNotAvailableError: Currency Rates Source Not Ready